In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from skopt import BayesSearchCV
from skopt.space import Integer, Real
import os
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../dataset/data.csv')
data = df.select_dtypes(include=['float64', 'int64'])
targets = ["Turbidity", "DO", "fChl"]
data.columns

Index(['Turbidity', 'DO', 'fChl', 'Discharge', 'Height', 'WT', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP',
       'MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI',
       '2BDA_Chl', 'RR'],
      dtype='object')

In [3]:
def rf(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = RandomForestRegressor(random_state=42, n_jobs=-1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    save_dir = "../models/rf/split"
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{target}.pkl")
    with open(save_path, "wb") as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y_test, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y_test, y_pred), 2)
    r2 = np.around(r2_score(y_test, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y_test), 2)

    print(f"\Random Forest Performance for {target} (Hold-out Split):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [4]:
for target in targets:
    rf(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/rf/split\Turbidity.pkl
\Random Forest Performance for Turbidity (Hold-out Split):
MAE  = 9.6
RMSE = 13.6
R²   = 81.57 %
MBE  = 4.02
Model saved to ../models/rf/split\DO.pkl
\Random Forest Performance for DO (Hold-out Split):
MAE  = 0.32
RMSE = 0.4
R²   = 96.81 %
MBE  = -0.05
Model saved to ../models/rf/split\fChl.pkl
\Random Forest Performance for fChl (Hold-out Split):
MAE  = 1.36
RMSE = 1.8
R²   = 86.04 %
MBE  = -0.35


In [5]:
def cvrf(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = RandomForestRegressor(random_state=42, n_jobs=-1)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(model, X, y, cv=kf, n_jobs=-1)
    
    model.fit(X, y)
    save_dir = "../models/rf/cross"
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{target}.pkl")
    with open(save_path, "wb") as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y, y_pred), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Random Forest Performance for {target} (5-Fold CV):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [6]:
for target in targets:
    cvrf(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/rf/cross\Turbidity.pkl
Random Forest Performance for Turbidity (5-Fold CV):
MAE  = 9.07
RMSE = 12.69
R²   = 86.99 %
MBE  = 0.33
Model saved to ../models/rf/cross\DO.pkl
Random Forest Performance for DO (5-Fold CV):
MAE  = 0.45
RMSE = 0.64
R²   = 90.22 %
MBE  = 0.01
Model saved to ../models/rf/cross\fChl.pkl
Random Forest Performance for fChl (5-Fold CV):
MAE  = 1.6
RMSE = 2.09
R²   = 79.61 %
MBE  = 0.01
